<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


In [21]:
import io
df = pd.read_csv('Apple240.csv', skiprows=0)

In [22]:
df[:10]

,Source,News,Movement
0,seekingalpha.com,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,1
1,seekingalpha.com,The areas to be concerned are that there are f...,1
2,seekingalpha.com,Amazon will soon resume selling the Apple TV (...,1
3,seekingalpha.com,"Facing questions from tech analysts, Apple (NA...",0
4,www.thestreet.com,Cramer and the AAP team say investors need to ...,0
5,seekingalpha.com,One day after Apple acknowledged that it has b...,1
6,www.thestreet.com,1 . -- U.S. stock futures turned lower on Tues...,1
7,seekingalpha.com,"Meanwhile, eight more lawsuits have been filed...",1
8,seekingalpha.com,Shares in Apple and several of its suppliers f...,1
9,www.thestreet.com,1 . -- U.S. stock futures traded slightly high...,0


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['News'].values, df.iloc[:, -1].values, test_size = 0.2, random_state = 0)

train_examples, train_labels = np.array(X_train), np.array(y_train)
test_examples, test_labels = np.array(X_test), np.array(y_test)


In [35]:
train_examples[:10]

array(['Q1 ’16: -4.1%, -6.1% Source: Refinitiv IBES consensus estimates as of 10/22/18 Summary/conclusion: S&P 500 earnings growth is expected to slow to 10% in calendar 2019, and the numbers have been telling investors that for quite some time, as this blog discussed here . I don’t think this is the reason behind this market downdraft that started on October 1 ’18, but we’ll see. China is a growing issue – their growth has slowed and that has to come home to roost in US earnings at some point if it hasn’t already. Companies give some idea of next year’s guidance starting with the 3rd quarter earnings reports every year, but the real or hard guidance occurs with the January-February conference calls that contain 4th quarter results. If we average the above 2019 results, the Tech sector – over the 4 quarters – is expected to average 8.2% EPS growth and 5% revenue growth, inline with the S&P 500’s expected average of 9% EPS growth and 5% revenue growth. As we lap the corporate income tax

In [36]:
train_labels[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [37]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 29251, test entries: 7313


In [38]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.2355471 ,  0.11719096, -0.19826788,  0.62281877, -0.49590316,
        -0.738948  ,  0.7471023 ,  0.61302024, -0.501993  ,  0.2771705 ,
         0.07472976,  0.25415528,  0.35520852, -0.36702234,  0.62300056,
        -0.7748881 , -0.56931186,  0.45307308,  0.5203001 , -0.03229528,
        -0.09448511, -0.35797906,  0.9466501 , -0.08701267,  0.12805209,
         0.23130086, -0.6522287 ,  0.44974676, -0.20601137, -0.37608024,
         0.21680853, -0.1402651 ,  0.773747  , -0.58659416, -0.07478948,
        -0.08432465,  0.37597057, -0.5652265 ,  0.03625645, -0.664764  ,
        -0.5988652 ,  0.03650454,  0.3608431 ,  0.5561065 , -0.47489214,
        -0.78675216, -0.35815898, -0.03418034, -0.09439875,  0.2157843 ],
       [ 0.20898803, -0.37149653,  0.1520238 ,  0.07758274,  0.06931368,
        -0.05553383, -0.02514357,  0.19448043, -0.05831357, -0.20240895,
        -0.28871888,  0.04763449,  0.11879189, -0.27784148,  0.5745588 ,
 

In [39]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
#classifier.fit(train_examples, train_labels)

In [17]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [ ]:
classifier.fit(hub_layer(train_examples), y_train)

In [50]:
test_val = np.array(hub_layer(test_examples))

print(test_val[:10])

[[ 2.82547116e-01 -2.34666765e-01 -2.14027941e-01 -4.65551205e-02
  -1.11681029e-01 -1.58526242e-01  1.72551587e-01  1.88051194e-01
  -1.42896399e-01  5.81438392e-02 -4.65896241e-02  2.55970299e-01
   2.59896100e-01 -2.60172904e-01  8.69553834e-02  1.76518798e-01
   2.52628215e-02  1.75248235e-01  2.75710374e-01  1.12865597e-01
  -4.64544492e-03 -2.11440157e-02  2.00407580e-01 -2.24034563e-01
  -4.72124666e-02  7.35704601e-02 -3.60729069e-01  7.35478252e-02
  -2.72688400e-02  2.22680885e-02  2.71626353e-01  1.92006350e-01
   3.24577615e-02 -6.74752742e-02  3.05076540e-01 -1.86266899e-02
   9.89515632e-02  7.47746006e-02 -1.14414521e-01  1.37996823e-01
  -9.00528431e-02  1.86133102e-01  5.17327413e-02  2.16062278e-01
   3.18861961e-01 -2.44425565e-01 -9.02021080e-02  4.17774841e-02
  -8.43402743e-03  1.35443002e-01]
 [ 2.62121499e-01  3.94896477e-01 -8.52603987e-02  1.85536131e-01
   2.11890507e-02 -5.37062883e-02  4.39397603e-01 -3.63258988e-01
  -3.06484759e-01  3.75550658e-01  1.4390

In [51]:
y_pred = classifier.predict(test_val)

In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(test_labels, y_pred)*100,"%")
print(confusion_matrix(test_labels, y_pred))

56.830302201558865 %
[[2390 1561]
 [1596 1766]]
